# Video Processing for BT Lab
This notebook to test risky code fragments
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [20]:
#import libraries
import os
import cv2
import json
import re
import datetime
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip


In [21]:
"""delete unwanted files"""
desired_path = r"E:\Christian\DI_centre_structured\DI_THERMAL_FLIR\3\2 meters\H\WOB\frames"
os.chdir(desired_path)
copied_files = [bmp_files for bmp_files in os.listdir() if "Copy" in bmp_files]
copied_files

[]

In [22]:
# for copy_file in copied_files:
#     os.remove(copy_file)

In [23]:
def convert_mp4_to_frames(mp4_file:str, output_dir:str):
    video = cv2.VideoCapture(mp4_file)
    os.makedirs(output_dir, exist_ok=True)
    frame_count = 0
    not_end_of_vid = True

    while not_end_of_vid:
        ret, frame = video.read()
        
        if not ret:
            not_end_of_vid = False
            break

        frame_file = os.path.join(output_dir, f"frame{frame_count:05d}.jpg")
        cv2.imwrite(frame_file, frame)
        frame_count += 1

    video.release()

    print(f"Conversion complete: {frame_count} frames saved in {output_dir}")

In [24]:
def count_frames(video_path: str) -> int:
    frame_count = 0
    still_has_frames = True
    video = cv2.VideoCapture(video_path)

    if not video.isOpened():
        print("Error opening video file")
        return 2
    
    while still_has_frames:
        ret, frame = video.read()
        if not ret:
            still_has_frames = False
        else:
            frame_count += 1
    
    video.release()
    return frame_count

In [38]:
def get_video_properties(mp4_file: str, is_real=True) -> list:
    try:
        video_properties = []
        video = cv2.VideoCapture(mp4_file)
        
        if is_real:
            number_of_frames = count_frames(mp4_file)
            old_fps = video.get(cv2.CAP_PROP_FPS)
        else:
            number_of_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
            old_fps = video.get(cv2.CAP_PROP_FPS)
        
        length = number_of_frames / old_fps
        video_time = datetime.timedelta(seconds=length)
        video.release()
    except Exception as e:
        print(f"{type(e)} - from {mp4_file}")
    else:
        video_properties.append(number_of_frames)
        video_properties.append(old_fps)
        video_properties.append(length)
        video_properties.append(video_time)
    
    return video_properties

In [33]:
def cut_video(input_file, output_file, tmp_video, start_time, end_time, target_fps):
    try:
        temp_file = tmp_video
        ffmpeg_extract_subclip(input_file, start_time, end_time, targetname=temp_file)
        clip = VideoFileClip(temp_file)
        final_clip = clip.set_fps(target_fps)
        final_clip.write_videofile(output_file, codec="libx264", audio_codec="aac")
        print("Video successfully cut!")
    except Exception as e:
        print(f"{type(e)} with {input_file}")

In [54]:
def fix_fps(input_file, output_file, target_fps):
    clip = VideoFileClip(input_file)
    final_clip = clip.set_fps(target_fps)
    final_clip.write_videofile(output_file)

In [57]:
""" raw values """

video_file = "E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/test/Arun2.mp4"

frames, fps, length_in_seconds, length_in_datetime = get_video_properties(video_file, is_real=False)

frames, fps, length_in_seconds, length_in_datetime

(143.0,
 18.563774028241603,
 7.703174999999999,
 datetime.timedelta(seconds=7, microseconds=703175))

In [48]:
""" correct values """

frames, fps, length_in_seconds, length_in_datetime = get_video_properties(video_file)

frames, fps, length_in_seconds, length_in_datetime

(139,
 18.563774028241603,
 7.487701573426572,
 datetime.timedelta(seconds=7, microseconds=487702))

In [55]:
""" edit video based on real values """
new_video = "E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/test/Arun2_1.mp4"

tmp_video = "E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/test/Arun2_tmp.mp4"

# cut_video(video_file, new_video, tmp_video, 0, length_in_seconds, fps)

""" without cutting video first"""
fix_fps(video_file, new_video, 18)

18
Moviepy - Building video E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/test/Arun2_1.mp4.
Moviepy - Writing video E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/test/Arun2_1.mp4



Moviepy - Done !
Moviepy - video ready E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/test/Arun2_1.mp4


In [56]:
""" rerunning get properties with is_real=False, expected output should be 18 fps and 139 frames """
frames, fps, length_in_seconds, length_in_datetime = get_video_properties(new_video, is_real=False)
frames, fps, length_in_seconds, length_in_datetime

(139.0,
 18.0,
 7.722222222222222,
 datetime.timedelta(seconds=7, microseconds=722222))

In [58]:
output_file = "E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/test/frames"
convert_mp4_to_frames(new_video, output_file)

Conversion complete: 139 frames saved in E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/test/frames
